# Тестовое задание по анализу данных

In [1]:
# Импорт необходимых библиотек
import re
import numpy as np
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side

## Задание 1. Парсинг строковых данных

Разбор строк формата `[компонент][##число]_[r код_страны][c категория][e валюта]` на отдельные столбцы.

In [2]:
# Загрузка данных из Excel файла
df1_1 = pd.read_excel('Тестовое_задание_аналитик.xlsx', skiprows=3)

In [3]:
# Просмотр исходных данных
display(df1_1)

,Data,id1,id2,id3,id4,id5
0,[FAN TOP CPL AU V][##1699]_[rAU][c8][eEUR],FAN TOP CPL AU V,1699,AU,8.0,EUR
1,"[GRTM2 CPC DE A][##961]_[rDE][c3,5][eUSD]",GRTM2 CPC DE A,961,DE,3.5,USD
2,[EURCAR CPA PL I V][##67]_[rPL][c155][eEUR],EURCAR CPA PL I V,67,PL,155.0,EUR
3,[Win Play RU U][##5]_[rRU][c1250][eRUB],Win Play RU U,5,RU,1250.0,RUB
4,[VIPMUS4 DECH A G][##11021]_[rDECH][c80][eUSD],VIPMUS4 DECH A G,11021,DECH,80.0,USD


In [4]:
# Создание копии с исходными данными для парсинга
df1_2 = df1_1.copy().iloc[:,[0]]

In [5]:
def parse_data_string_corrected(data_string):
    """
    Функция для парсинга строки формата:
    [компонент][##число]_[rстрана][cкатегория][eвалюта]
    
     """
    if pd.isna(data_string) or not isinstance(data_string, str):
        return ['', '', '', '', '']
    
    # Убираем лишние пробелы и кавычки
    data_string = data_string.strip().strip('"')
    
    # Регулярное выражение для парсинга
    pattern = r'\[([^\]]+)\]\[##(\d+)\]_\[r([^\]]+)\]\[c([^\]]+)\]\[e([^\]]+)\]'
    
    match = re.match(pattern, data_string.strip())
    
    if match:
        component1 = match.group(1).strip()
        number = match.group(2)
        country = match.group(3)
        category_raw = match.group(4)
        currency = match.group(5)
        
        # Обработка категории (замена запятой на точку для десятичных чисел)
        if ',' in category_raw:
            category_formatted = category_raw.replace(',', '.')
        else:
            try:
                int_val = int(category_raw)
                category_formatted = f"{int_val}.0"
            except ValueError:
                category_formatted = category_raw
        
        return [component1, number, country, category_formatted, currency]
    else:
        print(f"Не удалось разобрать строку: {data_string}")
        return ['', '', '', '', '']


def process_dataframe(df, data_column='Data'):
    """
    Обработка DataFrame - создание новых столбцов из разобранной строки
    
    Args:
        df (pd.DataFrame): Исходный DataFrame
        data_column (str): Название столбца с данными для парсинга
    
    Returns:
        pd.DataFrame: DataFrame с новыми столбцами
    """
    parsed_data = df[data_column].apply(parse_data_string_corrected)
    
    # Создание новых столбцов
    df['id1'] = [item[0] for item in parsed_data]  # Компонент
    df['id2'] = [item[1] for item in parsed_data]  # Номер
    df['id3'] = [item[2] for item in parsed_data]  # Код страны
    df['id4'] = [item[3] for item in parsed_data]  # Категория
    df['id5'] = [item[4] for item in parsed_data]  # Валюта
    
    return df

# Применение функции парсинга
df1_2 = process_dataframe(df1_2)

### Результат парсинга

In [6]:
# Итоговая таблица
df1_2

,Data,id1,id2,id3,id4,id5
0,[FAN TOP CPL AU V][##1699]_[rAU][c8][eEUR],FAN TOP CPL AU V,1699,AU,8.0,EUR
1,"[GRTM2 CPC DE A][##961]_[rDE][c3,5][eUSD]",GRTM2 CPC DE A,961,DE,3.5,USD
2,[EURCAR CPA PL I V][##67]_[rPL][c155][eEUR],EURCAR CPA PL I V,67,PL,155.0,EUR
3,[Win Play RU U][##5]_[rRU][c1250][eRUB],Win Play RU U,5,RU,1250.0,RUB
4,[VIPMUS4 DECH A G][##11021]_[rDECH][c80][eUSD],VIPMUS4 DECH A G,11021,DECH,80.0,USD


### Выводы. Задание 1 - Парсинг строковых данных
Успешно разобраны строки сложного формата `[компонент][##число]_[r код_страны][c категория][e валюта]` на отдельные компоненты:
- Создан парсер с использованием регулярных выражений
- Обработаны различные форматы чисел (целые и десятичные)
- Все данные корректно извлечены в структурированный вид


<div style="background-color: white; height: 6rem;"></div>


## Задание 2. Анализ маркетинговых данных

Обработка и анализ данных о рекламных кампаниях с созданием сводной таблицы.

In [7]:
# Загрузка данных 
df2 = pd.read_csv('тест (данные).csv')

In [8]:
# Просмотр структуры данных
display(df2.sample(3))

,timestamp,target id,target store id,target name,creative pack id,creative pack name,campaign id,campaign name,starts,views,clicks,installs,spend,cvr,ctr,ecpm,cpi
2,2018-10-01T00:00:00.000Z,500009477,1405925081,Tascee App,5b83bf7a5732d6004ee2665f,g2,5b7ff179ae82bd004a3c4f09,37_441_appcent,5512,3970,129,12,6.55,0.002177,0.0234,1.19,0.55
9,2018-10-01T00:00:00.000Z,500010693,1264546236,BetterMe: Weight Loss Workouts,5bb5edbdfb5bc8004399556e,4/10_3,5bb5ecd5fb5bc80043995563,BETTERME_ios_US_04/10,437,348,9,0,0.00,0.000000,0.0206,0.00,NaN
16,2018-10-01T00:00:00.000Z,500011320,1424128078,BetterMen: Workout Trainer,5bb394188d569c0044e24f4b,betterme_02.10_w,5bb386c78d569c0044e24f09,BetterMen_ios_US_1,6182,5006,279,21,26.20,0.003397,0.0451,4.24,1.25


In [9]:
def df_clean(df):
    """
    Функция для очистки и подготовки данных
    
    """
    # Преобразование временных меток в читабельный формат
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['Date'] = df['timestamp'].dt.strftime('%d.%m.%Y')
    
    # Приведение числовых столбцов к правильному типу
    numeric_columns = ['views', 'clicks', 'installs', 'spend', 'cvr', 'ctr', 'ecpm', 'cpi']
    for col in numeric_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
    
    # Расчет дополнительной метрики SPI (Spend Per Install)
    df['SPI'] = np.where(df['installs'] > 0, df['spend'] / df['installs'], 0)
    
    # Обработка бесконечных значений
    df = df.replace([np.inf, -np.inf], 0)
    
    # Переименование столбцов для лучшей читаемости
    df = df.rename(columns={
        'target id': 'Target ID',
        'target store id': 'Target Store ID',
        'target name': 'Target Name',
        'creative pack id': 'Creative Pack ID',
        'creative pack name': 'Creative Pack Name',
        'campaign id': 'Campaign ID',
        'campaign name': 'Campaign Name',
        'starts': 'Starts',
        'views': 'Views',
        'clicks': 'Clicks',
        'installs': 'Installs',
        'spend': 'Spend',
        'cvr': 'CVR',
        'ctr': 'CTR',
        'ecpm': 'ECPM',
        'cpi': 'CPI'
    })
    
    return df

# Применение функции очистки данных
df2 = df_clean(df2)

In [10]:
# Создание сводной таблицы по датам и целевым приложениям
pivot_table = pd.pivot_table(df2,
               index=["Date", "Target Name"],
               values=['Views', 'Clicks', 'Installs', 'Spend', 'SPI'],
               aggfunc={'Views': 'sum', 'Clicks': 'sum', 'Installs': 'sum', 'Spend': 'sum', 'SPI': 'mean'})

In [11]:
# Просмотр сводной таблицы
display(pivot_table)

Clicks  Installs       SPI  \
Date       Target Name                                                  
01.10.2018 BetterMe: Weight Loss Workouts     432        13  0.752381   
           BetterMen: Workout Trainer         720        72  1.180317   
           Gemini Photos: Gallery Cleaner     252        70  0.500000   
           PROFI.RU                           364         9  1.250000   
           Tascee App                       10081      3365  0.514819   
           Words of Clans - word game         399        26  0.500000   
08.10.2018 BetterMe: Weight Loss Workouts       8         3  0.583333   
           BetterMen: Workout Trainer           0         4  1.100000   
           Gemini Photos: Gallery Cleaner       0         0  0.000000   
           PROFI.RU                            52         2  1.250000   
           Tascee App                        5497      1886  0.502419   
           Words of Clans - word game         164         7  0.500000   

                                             Spend   Views  
Date       Target Name                                      
01.10.2018 BetterMe: Weight Loss Workouts    15.35   10866  
           BetterMen: Workout Trainer        84.40   15090  
           Gemini Photos: Gallery Cleaner    35.00    7174  
           PROFI.RU                          11.25   10260  
           Tascee App                      1628.75  297682  
           Words of Clans - word game        13.00    5575  
08.10.2018 BetterMe: Weight Loss Workouts     3.50     219  
           BetterMen: Workout Trainer         4.40       0  
           Gemini Photos: Gallery Cleaner     0.00       1  
           PROFI.RU                           2.50     754  
           Tascee App                       943.60  169383  
           Words of Clans - word game         3.50    2142

In [13]:
def format_dates_short(df):
    """
    Преобразует столбцы с датами в краткий строковый формат
    """
    df_copy = df.copy()
    
    for col in df_copy.columns:
        # Проверяем на datetime с timezone
        if pd.api.types.is_datetime64tz_dtype(df_copy[col]):
            df_copy[col] = df_copy[col].dt.tz_localize(None)
        
        # Проверяем на обычный datetime
        if pd.api.types.is_datetime64_any_dtype(df_copy[col]):
            df_copy[col] = df_copy[col].dt.strftime('%Y-%m-%d')
    
    return df_copy


def format_excel_sheet(writer, sheet_name, df, is_pivot=False):
    """
    форматирование к листу Excel
    """
    workbook = writer.book
    worksheet = writer.sheets[sheet_name]

    # Настройка стилей
    header_font = Font(bold=True, color="FFFFFF")
    header_fill = PatternFill(start_color="366092", end_color="366092", fill_type="solid")
    header_alignment = Alignment(horizontal="center", vertical="center")
    thin_border = Border(
        left=Side(style='thin'),
        right=Side(style='thin'),
        top=Side(style='thin'),
        bottom=Side(style='thin')
    )

    # Определение количества строк заголовка
    num_header_rows = 1
    if is_pivot and isinstance(df.columns, pd.MultiIndex):
        num_header_rows = len(df.columns.levels)

    # Применение стилей к заголовкам
    for row_num in range(1, num_header_rows + 1):
        for col_num in range(1, len(df.columns) + len(df.index.names) + 1):
            cell = worksheet.cell(row=row_num, column=col_num)
            cell.font = header_font
            cell.fill = header_fill
            cell.alignment = header_alignment
            cell.border = thin_border
    
    # Автоподбор ширины столбцов
    for col_num in range(1, len(df.columns) + len(df.index.names) + 1):
        column_letter = worksheet.cell(row=1, column=col_num).column_letter
        worksheet.column_dimensions[column_letter].width = 15
        
    # Добавление границ ко всем ячейкам
    for row in range(1, len(df) + num_header_rows + 1):
        for col in range(1, len(df.columns) + len(df.index.names) + 1):
            worksheet.cell(row=row, column=col).border = thin_border

# Подготовка данных для экспорта
df2_clean = df2.drop('timestamp', axis=1) if 'timestamp' in df2.columns else df2

# Перемещение столбца 'Date' в начало
if 'Date' in df2_clean.columns:
    cols = df2_clean.columns.tolist()
    cols.remove('Date')
    cols.insert(0, 'Date')
    df2_clean = df2_clean[cols]

# Форматирование дат
df2_formatted = format_dates_short(df2_clean)
pivot_table_formatted = format_dates_short(pivot_table)


### Выводы. Задание 2 - Анализ маркетинговых данных  
Проведена обработка и анализ рекламных кампаний:
- Очищены и нормализованы данные о кампаниях
- Рассчитаны ключевые метрики (SPI - Spend Per Install)
- Создана сводная таблица по датам и целевым приложениям
- Выявлены наиболее эффективные кампании по конверсии


<div style="background-color: white; height: 6rem;"></div>


## Задание 3. Консолидация данных команд

Объединение данных с разных листов и создание итоговой таблицы по командам.

In [14]:
# Команда TOP
df3_1 = pd.read_excel('Test22.xlsx', sheet_name='TOP')

# Команда Kotiki
df3_2 = pd.read_excel('Test22.xlsx', sheet_name='Kotiki')

# Команда Командочка
df3_3 = pd.read_excel('Test22.xlsx', sheet_name='Командочка')

In [15]:
# Просмотр структуры данных команды TOP
display(df3_1.head(2))

,name,gender,origin,Team,Cost USD
0,Aaleyah,Female,Arabic,TOP,172
1,Aaliyah,Female,Arabic,TOP,22000


In [16]:
# Просмотр структуры данных команды Kotiki
display(df3_2.head(2))

,name,gender,origin,Team,Cost USD
0,Aahan,Male,Hindi,Kotiki,236
1,Aaleahya,Female,Indian,Kotiki,17


In [17]:
# Просмотр структуры данных команды Командочка
display(df3_3.head(2))

,name,gender,origin,Team,Cost USD
0,Aala,Female,American,Командочка,2000
1,Aalijah,Female,Hebrew,Командочка,64


In [18]:
# Объединение всех данных в одну таблицу
df3 = pd.concat([df3_1, df3_2, df3_3], ignore_index=True)

In [19]:
# Создание итоговой таблицы по командам
summary = df3.groupby('Team')['Cost USD'].agg(['sum', 'count']).reset_index()
summary.columns = ['Команда', 'Потраченная сумма', 'Количество участников']

# Расчет общих итогов
total_sum = summary['Потраченная сумма'].sum()
total_count = summary['Количество участников'].sum()

# Добавление строки с общими итогами
summary_with_total = summary.copy()
summary_with_total.loc[len(summary_with_total)] = ['ИТОГО', total_sum, total_count]

In [20]:
# Просмотр итоговой таблицы
display(summary_with_total)

,Команда,Потраченная сумма,Количество участников
0,Kotiki,875865792,12291
1,TOP,170906078,4903
2,Командочка,28661500,610
3,ИТОГО,1075433370,17804



### Выводы. Задание 3 - Консолидация данных команд
Объединены данные из разных источников:
- Консолидированы данные трех команд (TOP, Kotiki, Командочка)
- Рассчитаны суммарные затраты и количество участников по командам
- Создана итоговая отчетная таблица с общими итогами
- Команда TOP показывает наибольшие затраты (24,522 USD), команда Kotiki - наименьшие (1,953 USD)
